## Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [ ]:
# Determine the number of unique values in each column.
unique_value_counts = application_df.nunique()
print(unique_value_counts)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [ ]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < 500].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_over1 = classification_counts[classification_counts > 1]
print(classification_counts_over1)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classification_cutoff = 1000
classifications_to_replace = classification_counts[classification_counts < classification_cutoff].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_variables = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']
application_df_encoded = pd.get_dummies(application_df, columns=categorical_variables)


In [ ]:
# Split our preprocessed data into our features and target arrays
X = application_df_encoded.drop('IS_SUCCESSFUL', axis=1)
y = application_df_encoded['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation="relu", input_dim=X_train_scaled.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 5)                 220       
                                                                 
 dense_3 (Dense)             (None, 5)                 30        
                                                                 
 dense_4 (Dense)             (None, 1)                 6         
                                                                 
Total params: 256 (1.00 KB)
Trainable params: 256 (1.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
model_history = nn.fit(X_train_scaled, y_train, epochs=100, verbose=1, validation_data=(X_test_scaled, y_test))


Epoch 1/100
804/804 [==============================] - 6s 5ms/step - loss: 0.6347 - accuracy: 0.6648 - val_loss: 0.5856 - val_accuracy: 0.7060
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5699 - accuracy: 0.7226 - val_loss: 0.5699 - val_accuracy: 0.7158
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5618 - accuracy: 0.7276 - val_loss: 0.5672 - val_accuracy: 0.7166
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5593 - accuracy: 0.7306 - val_loss: 0.5655 - val_accuracy: 0.7184
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5579 - accuracy: 0.7316 - val_loss: 0.5653 - val_accuracy: 0.7201
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5569 - accuracy: 0.7316 - val_loss: 0.5629 - val_accuracy: 0.7193
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5560 - accuracy: 0.7319 - val_loss: 0.5633 - val_accuracy: 0.7187

In [ ]:
# Train the model attempt 2
model_history = nn.fit(X_train_scaled, y_train, epochs=250, verbose=1, validation_data=(X_test_scaled, y_test))

Epoch 1/250
804/804 [==============================] - 3s 4ms/step - loss: 0.5483 - accuracy: 0.7342 - val_loss: 0.5570 - val_accuracy: 0.7212
Epoch 2/250
804/804 [==============================] - 2s 3ms/step - loss: 0.5481 - accuracy: 0.7345 - val_loss: 0.5568 - val_accuracy: 0.7217
Epoch 3/250
804/804 [==============================] - 2s 3ms/step - loss: 0.5481 - accuracy: 0.7338 - val_loss: 0.5575 - val_accuracy: 0.7215
Epoch 4/250
804/804 [==============================] - 2s 3ms/step - loss: 0.5482 - accuracy: 0.7346 - val_loss: 0.5568 - val_accuracy: 0.7227
Epoch 5/250
804/804 [==============================] - 3s 3ms/step - loss: 0.5479 - accuracy: 0.7337 - val_loss: 0.5572 - val_accuracy: 0.7216
Epoch 6/250
804/804 [==============================] - 2s 3ms/step - loss: 0.5481 - accuracy: 0.7343 - val_loss: 0.5564 - val_accuracy: 0.7226
Epoch 7/250
804/804 [==============================] - 2s 3ms/step - loss: 0.5480 - accuracy: 0.7345 - val_loss: 0.5570 - val_accuracy: 0.7220

In [ ]:
# Train the model attempt 3
model_history = nn.fit(X_train_scaled, y_train, epochs=350, verbose=1, validation_data=(X_test_scaled, y_test))

Epoch 1/350
804/804 [==============================] - 2s 3ms/step - loss: 0.5460 - accuracy: 0.7345 - val_loss: 0.5561 - val_accuracy: 0.7206
Epoch 2/350
804/804 [==============================] - 2s 3ms/step - loss: 0.5459 - accuracy: 0.7349 - val_loss: 0.5568 - val_accuracy: 0.7222
Epoch 3/350
804/804 [==============================] - 3s 4ms/step - loss: 0.5458 - accuracy: 0.7353 - val_loss: 0.5565 - val_accuracy: 0.7214
Epoch 4/350
804/804 [==============================] - 2s 3ms/step - loss: 0.5457 - accuracy: 0.7352 - val_loss: 0.5566 - val_accuracy: 0.7229
Epoch 5/350
804/804 [==============================] - 2s 2ms/step - loss: 0.5456 - accuracy: 0.7352 - val_loss: 0.5563 - val_accuracy: 0.7230
Epoch 6/350
804/804 [==============================] - 2s 3ms/step - loss: 0.5456 - accuracy: 0.7347 - val_loss: 0.5573 - val_accuracy: 0.7226
Epoch 7/350
804/804 [==============================] - 2s 2ms/step - loss: 0.5454 - accuracy: 0.7353 - val_loss: 0.5565 - val_accuracy: 0.7228

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5575 - accuracy: 0.7199 - 368ms/epoch - 1ms/step
Loss: 0.5574548244476318, Accuracy: 0.719883382320404


In [ ]:
# Export our model to HDF5 file
AlphabetSoupCharity = "Downloads/deep-learning-challenge/deep-learning/AlphabetSoupCharity.h5"
nn.save(AlphabetSoupCharity)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
